In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

# Define model_ckpt with the name of a pretrained model
model_ckpt = "bert-base-uncased"  # Example: You can replace this with your desired model

num_labels=6

# Detect if a GPU is available and assign it to device. Otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=(AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=num_labels).to(device))

In [ ]:
from sklearn.metrics import accuracy_score,f1_score
def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)
  f1=f1_score(labels,preds,average="weighted")
  acc=accuracy_score(labels,preds)
  return {"accuracy":acc,"f1":f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
batch_size = 64
# Ensure emotions_encoded is defined and contains the "train" key
# Example:
# emotions_encoded = {"train": your_training_data, "test": your_test_data}
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
 num_train_epochs=2,
 learning_rate=2e-5,
 per_device_train_batch_size=batch_size,
 per_device_eval_batch_size=batch_size,
 weight_decay=0.01,
 evaluation_strategy="epoch",
 disable_tqdm=False,
 logging_steps=logging_steps,
 push_to_hub=True,
 log_level="error")

In [ ]:
from transformers import Trainer
trainer = Trainer(model=model, args=training_args,
 compute_metrics=compute_metrics,
 train_dataset=emotions_encoded["train"],
 eval_dataset=emotions_encoded["validation"],
 tokenizer=tokenizer)
trainer.train();


In [ ]:
preds_output.metrics


In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)


In [ ]:
plot_confusion_matrix(y_preds, y_valid, labels)